In [1]:
# Load DataFrame and Libs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

df = pd.read_csv('Dataframes/3rdBodyPerturbationPoliastro.csv')
df_v = pd.read_csv('Dataframes/3rdBodyPerturbationPoliastro_vv.csv')

df['x_vv']=df_v.vv_x
df['y_vv']=df_v.vv_y
df['z_vv']=df_v.vv_z

In [2]:
# Full 6D r,v elements

df

,x,y,z,x_vv,y_vv,z_vv
0,42159.783600,0.000000,0.000000,0.000000,3.074505,0.053666
1,39159.545914,15563.916303,266.913399,-1.142984,2.852848,0.048528
2,30572.229294,28909.053086,496.928223,-2.127756,2.228986,0.039518
3,17596.068412,38144.740319,671.356184,-2.813645,1.287415,0.027234
4,2068.446366,41949.706443,775.646554,-3.098444,0.161675,0.012698
...,...,...,...,...,...,...
995,38132.961517,21370.880108,-7532.333000,-0.876644,2.199413,1.750026
996,31399.515894,31233.289707,1828.575655,-1.692745,1.562118,1.820311
997,20906.077447,37238.565267,10967.804062,-2.312211,0.726117,1.664753
998,7871.245318,38575.122415,18731.818721,-2.661184,-0.221639,1.293445


In [3]:
# convert an array of values into a dataset matrix

def create_dataset(dataset, look_back):
    dataX = []
    dataY = []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, predicted_feature]) # 2 stands for prediction of Z!!! (3rd col)
    return np.array(dataX), np.array(dataY)

In [4]:
# Function

def train_vector(df, predicted_feature):
    
    # load the dataset
    dataset = df.values
    
    # split into train and test sets
    train_size = int(len(dataset) * 0.8) 
    test_size = len(dataset) - train_size
    train=dataset[0 : train_size]
    test=dataset[train_size - 60 : ]
    
    #Scale the data
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    # reshape into X=t and Y=t+1
    look_back = 60 # this is the window
    trainX, trainY = create_dataset(train, look_back)  
    testX, testY = create_dataset(test, look_back)
    # reshape input to be  [length of train/test data, window size (def=60), features]
    trainX = np.reshape(trainX, (trainX.shape[0], look_back, 6))
    testX = np.reshape(testX, (testX.shape[0],look_back, 6))
    
    # Build the LSTM model
    # Note: replace LSTM with GRU or RNN if you want to try those
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (look_back, 6)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    # Train the model
    history=model.fit(trainX, trainY, batch_size=256, epochs=100)

    predictions = model.predict(testX)
    predict_ext = np.zeros((len(predictions), 6))
    predict_ext[:,predicted_feature] = predictions[:,0]
    predictions = scaler.inverse_transform(predict_ext)[:,predicted_feature]
    
    testY_extended = np.zeros((len(testY),6))
    testY_extended[:,predicted_feature]=testY
    testY=scaler.inverse_transform(testY_extended)[:,predicted_feature]
    
    rmse = np.sqrt(np.mean(((predictions - testY) ** 2)))
    print(f'RMSE for feature {predicted_feature} is: ' + str(rmse))
    
    pred=np.reshape(predictions, (predictions.shape[0]))
    pred_s=pd.Series(pred)
    df_pred[str(predicted_feature)]=pred_s
    
    #Output dataframe of predictions in 6 dimensions r,v
    return df_pred

In [5]:
# Call

train_vector(df, 0)

NameError: name 'predicted_feature' is not defined

In [ ]:
# Call for full 6D training

# Init empty predictions DF
df_pred= pd.DataFrame(columns=['0', '1', '2', '3', '4', '5'])

for predicted_feature in range(0,6):
    train_vector(df, predicted_feature)

#Rename predictions df columns from numerals
df_pred.columns=['x', 'y', 'z', 'x_vv', 'y_vv', 'z_vv']
df_pred

### Compare and plot results/errors SEPARATELY

In [ ]:
train_size = int(len(df) * 0.8) 

test_df = df[train_size:]
train_df = df[:train_size]

test_df

In [ ]:
df_pred.index=test_df.index

In [ ]:
for feature in list(test_df.columns):
    
    plt.figure(figsize=(16,8))

    plt.plot(train_df[feature])
    plt.plot(test_df[feature])
    plt.plot(df_pred[feature])

    plt.title(f'Feature {feature}')
    plt.xlabel('Time', fontsize=18)
    plt.ylabel('Coord [m]', fontsize=18)
    plt.legend(['Train', 'Test', 'Predictions'], loc='lower right')
    plt.show()


In [ ]:
# ABSOLUTE ERRORS:

for feature in list(test_df.columns):
    
    plt.figure(figsize=(16,8))

    plt.plot(abs(df_pred[feature]-test_df[feature])/max(test_df[feature]))

    plt.title(f'Feature {feature} absolute error plot:')
    plt.xlabel('Time', fontsize=18)
    plt.ylabel('Absolute Error [/]', fontsize=18)
    plt.show()

In [ ]:
# RELATIVE ERRORS

for feature in list(test_df.columns):
    
    plt.figure(figsize=(16,8))

    plt.plot(abs(df_pred[feature]-test_df[feature])/test_df[feature])

    plt.title(f'Feature {feature} relative error plot:')
    plt.xlabel('Time', fontsize=18)
    plt.ylabel('Relative Error [/]', fontsize=18)
    plt.show()

## Compare and Plot 3D trajectories/errors:

### 1. For 3D Position coordinates

In [ ]:
fig = plt.figure()
plt.figure(figsize=(50,20))

plt.axes(projection="3d")
x_line = df_pred['x']
y_line = df_pred['y']
z_line = df_pred['z']
plt.plot(x_line, y_line, z_line, 'red')

x_line2 = test_df['x']
y_line2 = test_df['y']
z_line2 = test_df['z']
plt.plot(x_line2, y_line2, z_line2, 'yellow')

plt.legend(['Predictions', 'Test'], loc='lower right')
plt.show()

### 2. For 3D velocity coordinates

In [ ]:
fig = plt.figure()
plt.figure(figsize=(50,20))

plt.axes(projection="3d")
x_line = df_pred['x_vv']
y_line = df_pred['y_vv']
z_line = df_pred['z_vv']
plt.plot(x_line, y_line, z_line, 'red')

x_line2 = test_df['x_vv']
y_line2 = test_df['y_vv']
z_line2 = test_df['z_vv']
plt.plot(x_line2, y_line2, z_line2, 'yellow')

plt.legend(['Predictions', 'Test'], loc='lower right')
plt.show()

## 3D Errors

### 1. Distance Error

https://www.engineeringtoolbox.com/distance-relationship-between-two-points-d_1854.html

In [ ]:
# Init empty DF for errors
df_err= pd.DataFrame(columns=['x_test', 'y_test', 'z_test', 'x_pred', 'y_pred', 'z_pred'])

df_err['x_test']=test_df['x']
df_err['y_test']=test_df['y']
df_err['z_test']=test_df['z']

df_err['x_pred']=df_pred['x']
df_err['y_pred']=df_pred['y']
df_err['z_pred']=df_pred['z']

df_err

In [ ]:
import math

# Difference in distance in 3D (Position)

df_err['distance_error'] = ( (df_err.x_test-df_err.x_pred)**2 + (df_err.y_test-df_err.y_pred)**2 + (df_err.z_test-df_err.z_pred)**2 )**(1/2)
df_err['distance_error'].apply(lambda x: float(x))
df_err

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df_err.distance_error)
plt.title('Distance errors for 3D position vectors')
plt.xlabel('Time', fontsize=18)
plt.ylabel('Distance Error (m)', fontsize=18)
plt.show()

### 2. Absolute errors

### Trying to make better/interactive plots :)

In [ ]:
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.io as pio

init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
fig = plt.figure()
plt.figure(figsize=(50,20))

plt.axes(projection="3d")
x_line = df_pred['x']
y_line = df_pred['y']
z_line = df_pred['z']
plt.iplot(x_line, y_line, z_line, 'red')

x_line2 = test_df['x']
y_line2 = test_df['y']
z_line2 = test_df['z']
plt.iplot(x_line2, y_line2, z_line2, 'yellow')

plt.legend(['Predictions', 'Test'], loc='lower right')
plt.show()

In [ ]:
import plotly.express as px

fig = px.line_3d(df_pred, x="x", y="y", z="z")
fig = px.line_3d(test_df, x="x", y="y", z="z")
fig.show()